In [ ]:
# default_exp segdata

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script

In [ ]:
#export
from fastai.vision.all import *
import skimage.io as skio
import warnings
warnings.filterwarnings("ignore")
import SSLRS.utils as utils
import rasterio
import slidingwindow as sw
from rasterio.windows import Window

In [ ]:
#export
src=rasterio.open('/home/staff/xin/Storage/SSLRS/data/32TQR/32tqr.vrt')
result=np.zeros((1,src.shape[0],src.shape[1]),dtype='uint8')
windows = sw.generate(result, sw.DimOrder.ChannelHeightWidth,512,0.6)

In [ ]:
names=[]
masks=[]
for window in windows:
        temp=src.read(11,window=Window(window.x,window.y,window.w,window.h))
        if np.count_nonzero(np.isnan(temp))==0:
            names.append(str(window))
            masks.append(str(window))
df=pd.DataFrame({'names':names,'masks':masks});
df.head()

,names,masks
0,"(205,6560,512,512)","(205,6560,512,512)"
1,"(205,6765,512,512)","(205,6765,512,512)"
2,"(205,6970,512,512)","(205,6970,512,512)"
3,"(205,7175,512,512)","(205,7175,512,512)"
4,"(205,7380,512,512)","(205,7380,512,512)"


In [ ]:
len(df)

3618

(10, 200, 200)

1
2
3
4
5
6
7
8
9
10
11


In [ ]:
#export

class MSTensorImage(TensorBase):   
    @classmethod
    def create(cls, data:(Path,str,ndarray), chnls=None):
        
        if isinstance(data, Path) or isinstance(data, str):
            ids=np.fromstring(data[1:-1],dtype=int,sep=',')
            with rasterio.open('/home/staff/xin/Storage/SSLRS/data/32TQR/32tqr.vrt') as src:
                im=src.read([i+1 for i in range(10)],window=Window(ids[0],ids[1],512,512))
            im=im.transpose(1,2,0).astype('float32')/10000

        elif isinstance(data, ndarray): 
            im = torch.from_numpy(data)
        else:
            im = data
        
        return im
class MSMask(TensorBase):
    @classmethod
    def create(cls, data:(Path,str,ndarray)):
        
        if isinstance(data, Path) or isinstance(data, str):
            ids=np.fromstring(data[1:-1],dtype=int,sep=',')
            with rasterio.open('/home/staff/xin/Storage/SSLRS/data/32TQR/32tqr.vrt') as src:
                im=src.read(11,window=Window(ids[0],ids[1],512,512))
                for i, blk in enumerate([10,20,30,40,50,60,70,80,90,95,100]):
                    im[im==blk]=i
        elif isinstance(data, ndarray): 
            im = torch.from_numpy(data)
        else:
            im = data
        
        return im

NameError: name 'TensorBase' is not defined

In [ ]:
db = DataBlock(blocks=(TransformBlock(type_tfms=partial(MSTensorImage.create)),
                       TransformBlock(type_tfms=partial(MSMask.create)),
                      ),
                get_x=ColReader('names'),
                get_y=ColReader('masks'),
               splitter=RandomSplitter(valid_pct=0.2)
              )

# db.summary(source=df)

In [ ]:
ds = db.datasets(source=df)

In [ ]:
#export
BAND_STATS = {
            'S2':{
                'mean': {
                    'B01': 340.76769064,
                    'B02': 429.9430203,
                    'B03': 614.21682446,
                    'B04': 590.23569706,
                    'B05': 950.68368468,
                    'B06': 1792.46290469,
                    'B07': 2075.46795189,
                    'B08': 2218.94553375,
                    'B8A': 2266.46036911,
                    'B09': 2246.0605464,
                    'B11': 1594.42694882,
                    'B12': 1009.32729131
                },
                'std': {
                    'B01': 554.81258967,
                    'B02': 572.41639287,
                    'B03': 582.87945694,
                    'B04': 675.88746967,
                    'B05': 729.89827633,
                    'B06': 1096.01480586,
                    'B07': 1273.45393088,
                    'B08': 1365.45589904,
                    'B8A': 1356.13789355,
                    'B09': 1302.3292881,
                    'B11': 1079.19066363,
                    'B12': 818.86747235
                }
            },
            'S1': {
                'mean': {
                    'VV': -12.619993741972035,
                    'VH': -19.29044597721542,
                    'VV/VH': 0.6525036195871579,
                },
                'std': {
                    'VV': 5.115911777546365,
                    'VH': 5.464428464912864,
                    'VV/VH': 30.75264076801808,
                },
                'min': {
                    'VV': -74.33214569091797,
                    'VH': -75.11137390136719,
                    'R': 3.21E-2
                },
                'max': {
                    'VV': 34.60696029663086,
                    'VH': 33.59768295288086,
                    'R': 1.08
                }
            }
        }


In [ ]:
#export
bands=['B02','B03', 'B04', 'B05','B06', 'B07', 'B11', 'B08','B8A', 'B12']
means=[BAND_STATS['S2']['mean'][band]/10000 for band in bands]
stds=[BAND_STATS['S2']['std'][band]/10000 for band in bands]

In [ ]:
#export
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from albumentations.core.transforms_interface import ImageOnlyTransform
import random


aug_pipe = A.Compose([A.ShiftScaleRotate(p=.5),
                      A.HorizontalFlip(),
                      A.Normalize(mean=means,std=stds,max_pixel_value=1.0),
                         ToTensorV2()]                    
                    )
val_pipe = A.Compose([
    A.Normalize(mean=means,std=stds,max_pixel_value=1.0),
                         ToTensorV2()]                    
                    )

class TrainTransform(ItemTransform):
    split_idx = 0
    def __init__(self, aug,split=0): 
        self.aug = aug
#         self.split_idx = split
    def encodes(self, x):
        aug = self.aug(image=x[0],mask=x[1])
#         print(torch.cat((aug['image0'],aug['image1']),axis=0).shape)
        return MSTensorImage(aug['image']), MSMask(aug['mask'])
class ValTransform(ItemTransform):
    split_idx = 1
    def __init__(self, aug,split=0): 
        self.aug = aug
#         self.split_idx = split
    def encodes(self, x):
        aug = self.aug(image=x[0],mask=x[1])
#         print(torch.cat((aug['image0'],aug['image1']),axis=0).shape)
        return MSTensorImage(aug['image']), MSMask(aug['mask'])

# Create our class with this aug_pipe
aug = TrainTransform(aug_pipe)
aug2=ValTransform(val_pipe)

In [ ]:
db = DataBlock(blocks=(TransformBlock(type_tfms=partial(MSTensorImage.create)),
                       TransformBlock(type_tfms=partial(MSMask.create)),
                      ),
               get_x=ColReader('names'),
                get_y=ColReader('masks'),
               item_tfms=[aug,aug2]
#                batch_tfms=transforms.Normalize(means,stds)
              )

In [ ]:
dls = db.dataloaders(source=df, bs=2,num_workers=8,pin_memory=True)

In [ ]:
aa,bb=first(dls.train)

In [ ]:
notebook2script(fname='./05_segdata.ipynb')

Converted 05_segdata.ipynb.
